1. Wstęp
- Importy bibliotek
- pobranie bazy ze zdjęciami
- import modelu

In [ ]:
import tensorflow as tf
import tensorflow_hub as hub
import numpy as np
import cv2
from tensorflow.keras.applications.resnet50 import ResNet50, preprocess_input, decode_predictions
from tensorflow.keras.models import Model
from matplotlib import pyplot as plt

2. Zasada działania Grad-cam

In [ ]:
def make_gradcam_heatmap(img_array, model, last_conv_layer_name, pred_index=None):
    # First, we create a model that maps the input image to the activations
    # of the last conv layer as well as the output predictions
    grad_model = tf.keras.models.Model(
        [model.inputs], [model.get_layer(last_conv_layer_name).output, model.output]
    )

    # Then, we compute the gradient of the top predicted class for our input image
    # with respect to the activations of the last conv layer
    with tf.GradientTape() as tape:
        last_conv_layer_output, preds = grad_model(img_array)
        if pred_index is None:
            pred_index = tf.argmax(preds[0])
        class_channel = preds[:, pred_index]

    # This is the gradient of the output neuron (top predicted or chosen)
    # with respect to the output feature map of the last conv layer
    grads = tape.gradient(class_channel, last_conv_layer_output)

    # This is a vector where each entry is the mean intensity of the gradient
    # over a specific feature map channel
    pooled_grads = tf.reduce_mean(grads, axis=(0, 1, 2))

    # We multiply each channel in the feature map array
    # by "how important this channel is" with respect to the top predicted class
    # then sum all the channels to obtain the heatmap class activation
    last_conv_layer_output = last_conv_layer_output[0]
    heatmap = last_conv_layer_output @ pooled_grads[..., tf.newaxis]
    heatmap = tf.squeeze(heatmap)

    # For visualization purpose, we will also normalize the heatmap between 0 & 1
    heatmap = tf.maximum(heatmap, 0) / tf.math.reduce_max(heatmap)
    return heatmap.numpy()

# Function to overlay a heatmap on an image
def overlay_heatmap(heatmap, image_np, alpha=0.4):
    if heatmap.shape != image_np.shape[0:2]:
        heatmap = cv2.resize(heatmap, (image_np.shape[1], image_np.shape[0]))
    heatmap = np.uint8(255 * heatmap)
    heatmap = cv2.applyColorMap(heatmap, cv2.COLORMAP_JET)
    superimposed_img = heatmap * alpha + image_np
    superimposed_img = np.clip(superimposed_img, 0, 255).astype(np.uint8)
    return superimposed_img

3. Omówienie wybranego modelu i zastosowanie go w Grad-cam

Wykorzystywany model dla Grad-cam: ResNet50

Dzieki wybranemu modelowi grad-cam może poprawnie określać pixele odpowiadające zwierzętom.

In [ ]:
classification_model = ResNet50(weights='imagenet')

Przykład zastosowania modelu w grad-cam na obrazie przedstawiającym dwa lwy.

In [ ]:
path_to_image = "lions.png"
image_example = cv2.imread(path_to_image)
image_example = cv2.cvtColor(image_example, cv2.COLOR_BGR2RGB)
image_example = cv2.resize(image_example, (224, 224))

image_example_preprocessed = preprocess_input(np.expand_dims(image_example, axis=0))
heatmap = make_gradcam_heatmap(image_example_preprocessed, classification_model, "conv5_block3_out")
overlayed_image = overlay_heatmap(heatmap, image_example)

plt.figure(figsize=(10, 15))
plt.imshow(overlayed_image)
plt.axis('off')
plt.show()

4. Połączenie Grad-cam (dla uproszczenia grad-cam z wybranym przez nas modelem będzie nazywany jako grad-cam) z modelem wykrywającym zwierzęta na obrazie.

Powodem takiego połąćzenia jest fakt iż grad-cam wykrywa zazwyczaj jeden konkretny element a podając mu model wykrywający i klasyfikujący wiele zaczyna działąć niepoprawnie.

Celem tego połączenia jest również wskazanie w jaki sposób grad cam przedstawi heatmap'ę dla wykrytych wcześniej wycinków obrazu.

Poniżej przedstawiony został kod programu.

In [ ]:
def animals_detection(path_to_image):
  detector_model_url = "https://tfhub.dev/tensorflow/efficientdet/lite4/detection/1"
  detector = hub.load(detector_model_url)

  # Load and preprocess the image
  image_path = path_to_image
  image_np = cv2.imread(image_path)
  image_np = cv2.cvtColor(image_np, cv2.COLOR_BGR2RGB)

  # Run object detection
  detections = detector(image_np[np.newaxis, ...])

  detection_boxes = detections[0].numpy()
  detection_classes = detections[2].numpy()
  detection_scores = detections[1].numpy()
  num_detections = int(detections[3][0].numpy())

  for i in range(num_detections):
      box = detection_boxes[0, i]
      class_id = int(detection_classes[0, i])
      score = detection_scores[0, i]

      image_height, image_width, _ = image_np.shape

      if score > 0.3:
          ymin, xmin, ymax, xmax = box
          ymin = int(ymin)
          xmin = int(xmin)
          ymax = int(ymax)
          xmax = int(xmax)

          # Skip the box if it has zero area
          print(xmin, xmax, ymax, ymin)
          if (xmax <= xmin) or (ymax <= ymin):
              continue

          # Crop and preprocess the image for the classification model
          crop_img = image_np[ymin:ymax, xmin:xmax]
          crop_img_resized = cv2.resize(crop_img, (224, 224))
          crop_img_preprocessed = preprocess_input(np.expand_dims(crop_img_resized, axis=0))

          # Get the classification label
          preds = classification_model.predict(crop_img_preprocessed)
          label = decode_predictions(preds, top=1)[0][0][1]

          # Get Grad-CAM heatmap
          heatmap = make_gradcam_heatmap(crop_img_preprocessed, classification_model, "conv5_block3_out")

          # Resize heatmap to the size of the crop
          heatmap_resized = cv2.resize(heatmap, (xmax - xmin, ymax - ymin))

          # Overlay heatmap on the original cropped image
          overlay_on_crop = overlay_heatmap(heatmap_resized, crop_img)

          plt.figure(figsize=(10, 15))
          plt.subplot(1, 3, 1)
          plt.imshow(crop_img)
          plt.title('Wycinek obrazu')
          plt.subplot(1, 3, 2)
          plt.title('Heatmap dla wycinka obrazu')
          plt.imshow(heatmap_resized)
          plt.axis('off')
          plt.subplot(1, 3, 3)
          plt.imshow(overlay_on_crop)
          plt.title('Heatmap nałożona na wycinek obrazu')
          plt.subplots_adjust(wspace=0.5)
          plt.show()

          # Replace the cropped area with the overlay heatmap
          image_np[ymin:ymax, xmin:xmax] = overlay_on_crop

  # Display the final image with bounding boxes and Grad-CAM heatmaps
  plt.figure(figsize=(10, 15))
  plt.imshow(image_np)
  plt.axis('off')
  plt.show()

5. Zadanie dla grupy - pradopodobnie samodzielne wybranie zdjęcia i wykorzystanie zaimplementowanych algorytmów w celu sprawdzenia modelu.

Przykładowe wybrane zdjęcie przedstawione zostało poniżej. W celu wykorzystania danego zdjęcia w progranie wymagane jest pobranie go a następnie podanie ścieżki do niego jako argument do funcji animals_detection(ścieżka).

In [ ]:
animals_detection("animals_2.png")

In [ ]:
layers = []
for layer in classification_model.layers:
    name = layer.name
    if len(name) > 2 and name[-3:] == "out":
      layers.append(name)

def show_image_on_layer_with_heatmap(image_path, layer):
  image = cv2.imread(image_path)
  image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
  image_fixed = cv2.resize(image, (224, 224))
  image_fixed = preprocess_input(np.expand_dims(image_fixed, axis=0))
  heatmap = make_gradcam_heatmap(image_fixed, classification_model, layer)
  heatmap_fixed = fix_heatmap(heatmap, image)
  show_image_heatmap_mixed(image, heatmap, layer)

def show_image_heatmap_mixed(image, heatmap, label):
  plt.figure(figsize=(10, 15))
  plt.subplot(1, 3, 1)
  plt.imshow(image)
  plt.title('obraz')
  plt.subplot(1, 3, 2)
  plt.title(f'Heatmap\nlayer: {label}')
  plt.imshow(heatmap)
  plt.axis('off')
  plt.subplot(1, 3, 3)
  plt.imshow(overlay_heatmap(heatmap, image))
  plt.title(f'obraz + heatmap\nlayer: {label}')
  plt.subplots_adjust(wspace=0.5)
  plt.show()

def fix_heatmap(heatmap, image):
  return cv2.resize(heatmap, (image.shape[0], image.shape[1]))

def show_all_layers_heatmap(image_path):
  image = cv2.imread(image_path)
  image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
  image_fixed = cv2.resize(image, (224, 224))
  image_fixed = preprocess_input(np.expand_dims(image_fixed, axis=0))
  for layer in classification_model.layers[2:]:
    try:
      heatmap = make_gradcam_heatmap(image_fixed, classification_model, layer.name)
    finally:
      heatmap_fixed = fix_heatmap(heatmap, image)
      show_image_heatmap_mixed(image, heatmap, layer.name)

show_all_layers_heatmap("lion.png")

Zadanie 1

Dla wybranego przez siebie obrazka wyświetl wszystkie możliwe heatmapy (wszystkie dla warst wyjściowych modelu klasyfikującego)

Zadanie 2

Wybierz najbardziej odpowiednią heatmapę dla wybranego przez siebie obrazka i wyświetl nałożenie heatmapy na obraz za pomocą funkcji